<a href="https://colab.research.google.com/github/maricamolesi/Iniciacao-Cientifica/blob/master/Descritores_imagens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Código para extrair descritores das imagens do dataset. Os descritores das imagens originais são concatenados com os descritores das imagens corroídas formando uma única matriz de descritores.

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image
from torchvision import datasets
from os import listdir
import os
from os.path import isfile, join
import numpy as np
import natsort 

In [19]:
#Carregar imagens
PATH_ORIGINAL = '/content/drive/My Drive/ProjetoIC/Brodatz/Brodatz_128x128'
LIST_ORIGINAL = natsort.natsorted([f for f in listdir(PATH_ORIGINAL) if isfile(join(PATH_ORIGINAL, f))],reverse=False)

PATH_CORRODED = '/content/drive/My Drive/ProjetoIC/v1'
LIST_CORRODED = natsort.natsorted([f for f in listdir(PATH_CORRODED) if isfile(join(PATH_CORRODED, f))],reverse=False)

FEATURES = []

#Carregar a rede pré treinada anteriormente (Notebook: Rede-neural)
model = torch.load('/content/drive/My Drive/ProjetoIC/Rede-Neural.pt', map_location='cpu')

#Escolha da camada que deseja extrair os descritores
layer = model._modules.get('avgpool')

#Definir modelo para o modo de avaliação
model.eval()


In [2]:
#Transformações na imagem
scaler = transforms.RandomResizedCrop(224)
flip = transforms.RandomHorizontalFlip()
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

In [ ]:
#Converter imagens em escala cinza para RGB
def pil_loader(path):
  with open(path, 'rb') as f:
    with Image.open(f) as img:
      return img.convert('RGB')

In [23]:
def get_vector(img):
    transformed_img = Variable(normalize(to_tensor(scaler(flip(img)))).unsqueeze(0))

    #Vetor de zeros para armazenar os descritores.
    #A camada 'avgpool' da rede Resnet50 possui saída de tamanho 2048
    my_embedding = torch.zeros(2048)

    #Função para copiar a saída da camada
    def copy_data(m, i, o):
       my_embedding.copy_(o.data.reshape(o.data.size(1)))

    h = layer.register_forward_hook(copy_data)
    model(transformed_img)
    h.remove()
    
    #Retorna vetor de descritores
    return my_embedding.numpy() 

In [ ]:
#Extraindo descritores
for n in range(1776):
  img_original = pil_loader(join(PATH_ORIGINAL, LIST_ORIGINAL[n]))
  img_corroded = pil_loader(join(PATH_CORRODED, LIST_CORRODED[n]))

  feature_original = get_vector(img_original)
  feature_corroded = get_vector(img_corroded)

  features = np.vstack([feature_original, feature_corroded]).reshape(1,4096)
  features = np.squeeze(features)

  FEATURES.append(features)

FEATURES = np.array(FEATURES)

In [ ]:
#Salvar matriz de descritores
np.save('/content/drive/My Drive/ProjetoIC/CITA/FEATURES_CORROIDO.npy', FEATURES)